In [10]:
import pandas as pd
import scipy
import numpy as np
from collections import defaultdict
from sklearn.naive_bayes import MultinomialNB

import data_processing as process
import naive_bayes as nb

dataset_name = 'domain_sentiment_data.tar.gz'
dataset_path = './data/'
dataset_url = 'http://www.cs.jhu.edu/~mdredze/datasets/sentiment/domain_sentiment_data.tar.gz'

dataset_generator = process.safely_create_datasets(dataset_name, dataset_path,
                                           dataset_url, process.read_xml)
grouped_datasets = process.group_datasets(dataset_generator)
df = process.unify_datasets(grouped_datasets)
text_features = process.vectorize_columns(
    df, columns=['title', 'review_text', 'product_name'])
# REVIEWER LOCATION TOO HARD TO PARSE AND IT'S INPUT SHOULD BE SMALL
df.drop(
    [
        'asin', 'date', 'reviewer', 'unique_id', 'product_type', 'title',
        'review_text', 'product_name', 'reviewer_location'
    ],
    axis=1,
    inplace=True)
_, codes = process.convert_to_numerical(
    df, columns=['category', 'review'], inplace=True)
df['rating'] = pd.to_numeric(df['rating'], errors='ignore')
process.reviews_to_numeric(df, ['helpful'], inplace=True)

# CONCATENATE SCIPY'S ARRAYS
# LEAVES CATEGORY AS IT'S VALUE IS ZERO
X = scipy.sparse.csr_matrix(df[:50].values)
X = scipy.sparse.hstack([text_features[:50,:], X]).tocsr()

X_training = X[:40,:-1]
Y_training = X[:40,-1]
Y_training[0,0] = 2
Y_training[1,0] = 2
Y_training[2,0] = 2
Y_training[3,0] = 2
X_test = X[40:50,:-1]
Y_test = X[40:50,-1]
clf = nb.sparse_naive_bayes().fit(X_training, Y_training)
clf2 = MultinomialNB().fit(X_training, np.ravel(Y_training))
clf2.predict(X_test)
clf.predict(X_test)


[[-13.86325155 -13.86325155 -13.86325155 ..., -13.19542218 -11.9173414
  -11.66602698]
 [-13.86778509 -13.86778509 -13.86778509 ..., -11.23141086  -9.93595945
   -9.57732564]]


ValueError: Found input variables with inconsistent numbers of samples: [40, 1]